In [3]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 25.0 MB/s eta 0:00:0000:0100:01


In [4]:
import numpy as np
import pandas as pd
import os
import cv2
import segmentation_models as sm
from tensorflow.keras import callbacks
from sklearn.model_selection import train_test_split

from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score

In [15]:
images = "/home/valerio/code/ValerioCann/ai4mars-dataset-merged-0.1/msl/images"
tr_lab = "/home/valerio/code/ValerioCann/ai4mars-dataset-merged-0.1/msl/labels/train"
#edr = '/home/valerio/code/ValerioCann/ai4mars-dataset-merged-0.1/msl/images/edr'
edr = images + "/edr"
mxy = images + "/mxy" # not required
rng = images + "/rng-30m" # not required

In [16]:
edr_files = os.listdir(edr)
trlab_files = os.listdir(tr_lab)

X = []
y = []
c = 0

# preparing X and y
for lab_name in trlab_files:
    img_name = lab_name[:-4] + ".JPG"
    
    if img_name in edr_files:
        
        img_path = os.path.join(edr, img_name)
        img_arr = cv2.imread(img_path)
        img_arr = cv2.resize(img_arr, dsize = (224, 224))
        
       
        lab_path = os.path.join(tr_lab, lab_name)
        lab_arr = cv2.imread(lab_path, 0)
        lab_arr = cv2.resize(lab_arr, (224, 224), interpolation = cv2.INTER_NEAREST)
        
        X.append(img_arr)
        y.append(lab_arr)
        
    c += 1
    if c >= 1000:
        break
        
X = np.asarray(X, dtype = np.float32) / 255.0
y = np.array(y, dtype = np.uint8)


# 0 - soil
# 1 - bedrock
# 2 - sand
# 3 - big rock
# 255 -> 4 - NULL (no label)


# keeping integer values in labels will help us in segmentation task (UNet)
y[y==255] = 4

print(X.shape, y.shape)

FileNotFoundError: [Errno 2] No such file or directory: '/home/valerio/code/ValerioCann/ai4mars-dataset-merged-0.1/msl/images/edr'

In [ ]:
X[255].shape

In [ ]:
plt.imshow(X[255])

In [ ]:
print(np.unique(y[255]))

plt.figure(figsize = (10, 10))

plt.imshow(X[255])
plt.imshow(y[255], alpha = 0.1)

plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 42)

In [ ]:
# Charger le modèle UNet avec l'encodeur ResNet50 pré-entraîné et des poids ImageNet
model = sm.Unet('resnet50', encoder_weights='imagenet', classes=5, activation='softmax', input_shape=X[255].shape[-1])) # input_shape=(255, 255, 3)

In [ ]:
model.compile(optimizer='adam', loss=sm.losses.bce_jaccard_loss, metrics=[sm.metrics.iou_score])

In [ ]:
model.summary()

In [ ]:
es = callbacks.EarlyStopping(patience=2, restore_best_weights=True)

history = model.fit(X_train, y_train,
          batch_size=16,
          epochs=3,
          validation_split=0.3,
          callbacks=[es],
          verbose=0)   

In [ ]:
def plot_history(history):
    plt.plot(history.history['loss'])
    plt.title('Train loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.show()
    
plot_history(history)

In [ ]:
model.evaluate(X_test, y_test)